https://www.youtube.com/watch?v=l67Lgk9A6wQ&t=354s

In [1]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys

Using TensorFlow backend.


In [2]:
fp = codecs.open("./07_data/BEXX0003.txt", "r", encoding="utf-16")
soup = BeautifulSoup(fp, "html.parser")
body = soup.select_one("body")
text = body.getText() + " "
print('코퍼스의 길이: ', len(text))

코퍼스의 길이:  311682


In [4]:
chars = sorted(list(set(text)))

In [7]:
chars

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '?',
 '[',
 ']',
 '×',
 '…',
 '○',
 '一',
 '丈',
 '三',
 '上',
 '下',
 '世',
 '中',
 '主',
 '久',
 '乙',
 '乞',
 '也',
 '乳',
 '事',
 '五',
 '亡',
 '亭',
 '人',
 '他',
 '代',
 '令',
 '仰',
 '位',
 '佛',
 '作',
 '使',
 '侍',
 '供',
 '修',
 '俵',
 '倒',
 '偉',
 '傘',
 '傳',
 '債',
 '像',
 '僕',
 '先',
 '光',
 '免',
 '兒',
 '公',
 '兼',
 '冥',
 '凄',
 '判',
 '制',
 '則',
 '加',
 '北',
 '千',
 '南',
 '參',
 '口',
 '古',
 '同',
 '名',
 '呪',
 '命',
 '商',
 '問',
 '啓',
 '喪',
 '嘲',
 '四',
 '回',
 '因',
 '國',
 '土',
 '地',
 '執',
 '堂',
 '壺',
 '壽',
 '外',
 '夜',
 '大',
 '天',
 '太',
 '央',
 '女',
 '奴',
 '妃',
 '妄',
 '姜',
 '姬',
 '婚',
 '婦',
 '子',
 '孫',
 '學',
 '安',
 '官',
 '家',
 '寒',
 '實',
 '將',
 '小',
 '屈',
 '屍',
 '山',
 '崔',
 '巡',
 '工',
 '巫',
 '巾',
 '布',
 '帝',
 '常',
 '干',
 '平',
 '年',
 '序',
 '底',
 '府',
 '庵',
 '廣',
 '廳',
 '弊',
 '引',
 '弟',
 '後',
 '得',
 '從',
 '德',
 '忌',
 '性',
 '怨',
 '患',
 '情',
 '意',
 '愚',
 '態',
 '懸',
 '戰',
 '手',
 '打',
 '按',
 '授',
 '接',
 '擧

In [8]:
char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자

In [12]:
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')

학습할 구문의 수: 103888
텍스트를 ID 벡터로 변환합니다...


In [17]:
sentences[:5], next_chars[:5]

(['\n\n제1편 어둠의 발소리\n서(序)\n1',
  '1편 어둠의 발소리\n서(序)\n1897',
  '어둠의 발소리\n서(序)\n1897년의 ',
  ' 발소리\n서(序)\n1897년의 한가위',
  '리\n서(序)\n1897년의 한가위.\n까'],
 ['8', '년', '한', '.', '치'])

In [18]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [19]:
# 모델 구축하기(LSTM)
print('모델을 구축합니다...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

모델을 구축합니다...


In [20]:
# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1) # 
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)
        # 시드를 기반으로 텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
반복 = 1


/Users/csbg/miniconda3/lib/python3.5/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
103888/103888 [==============================] - 366s - loss: 3.8284      

KeyboardInterrupt: 